In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, Lambda, Dropout, MaxPool2D, concatenate
from keras.layers import Conv2DTranspose, Concatenate, multiply, MaxPooling2D, UpSampling2D
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import MeanIoU
import tensorflow as tf
import keras.backend as K
import numpy as np
import os
import pandas as pd
import tifffile as tiff
import csv
import sys
import cv2
import shapely.wkt
import shapely.affinity
from skimage.segmentation import find_boundaries

Using TensorFlow backend.


In [2]:
Patch_size, N_split, Class_Type = 224, 15, 3

In [3]:
processed_data_dir = "preprocessed_data"

In [16]:
data_path = '..'
w0 = 10
sigma = 5
N_split = 15
Patch_size = 224
class_type = 1
class_dict = {class_type:0}
grid_sizes_df = pd.read_csv(os.path.join(data_path, 'grid_sizes.csv'))
grid_sizes_df = grid_sizes_df.rename(columns = {grid_sizes_df.columns[0]: 'IM_ID'})
polygons_df = pd.read_csv(os.path.join(data_path, 'train_wkt_v4.csv'))
show_mask = lambda mask: tiff.imshow(255*np.stack([mask, mask, mask]))

In [5]:
def getGridSize(img_id):
        return tuple(grid_sizes_df[grid_sizes_df['IM_ID'] == img_id][['Xmax' ,'Ymin']].values[0])
    
def getPolygons(img_id):
    poly_dict = dict()
    img_poly_df = polygons_df[polygons_df['ImageId'] == img_id]
    classes = img_poly_df['ClassType'].unique()
    for class_type in classes:
        poly_dict[int(class_type)] = shapely.wkt.loads(img_poly_df[img_poly_df['ClassType'] == class_type]['MultipolygonWKT'].values[0])
    return poly_dict

def getImage(img_id, img_band):

    if img_band =='RGB':
        file = os.path.join(data_path,'three_band', '{}.tif'.format(img_id))
    else:
        file = os.path.join(data_path,'sixteen_band', '{}_{}.tif'.format(img_id, img_band))    
    img = tiff.imread(file)
    img = img[:,:,None] if img_band == 'P' else np.rollaxis(img, 0, 3)
    img = img.astype(np.float32)/16384 if img_band == 'A' else img.astype(np.float32)/2048
    return img

def getImageAllband(img_id, Scale_Size = None):
    if not Scale_Size:
        Scale_Size = Patch_size * N_split
    img_RGB = cv2.resize(getImage(img_id, 'RGB'), (Scale_Size, Scale_Size))    
    img_M = cv2.resize(getImage(img_id, 'M'), (Scale_Size, Scale_Size))
    img_A = cv2.resize(getImage(img_id, 'A'), (Scale_Size, Scale_Size))
    img_P = cv2.resize(getImage(img_id, 'P'),( Scale_Size, Scale_Size))
    img = np.concatenate((img_RGB,img_M, img_A,img_P[:,:,None]), axis=2)
    return img

In [6]:
def getMask(img_id, img_shape):
    polygons_vals_all_classes = getPolygons(img_id)
    Xmax, Ymin = getGridSize(img_id)
    H, W = img_shape
    xfact = W * (W / (W + 1)) / Xmax
    yfact = H * (H / (H + 1)) / Ymin
    total_classes = len(class_dict)
    masks = np.zeros((H, W, total_classes), dtype=np.uint8)
    for class_type, polygons_vals in polygons_vals_all_classes.items():
        if class_type in class_dict:
            class_mask = np.zeros((H, W), dtype=np.uint8)
            polygon_img = shapely.affinity.scale(polygons_vals, xfact = xfact, yfact = yfact, origin = (0, 0, 0))
            if not polygon_img:
                continue
            external_region = []
            internal_region = []
            for poly_val in polygon_img:
                external_region.append(np.array(poly_val.exterior.coords).round().astype(np.int32))
                for poly_int in poly_val.interiors:
                    internal_region.append(np.array(poly_int.coords).round().astype(np.int32))
            cv2.fillPoly(class_mask, external_region, 1)
            cv2.fillPoly(class_mask, internal_region, 0)
            masks[:, :, class_dict[class_type]] = class_mask
    return masks

In [7]:
def getWeightMap(masks):
    nrows, ncols = masks.shape[1:]
    #masks = (masks > 0).astype(int)
    dist_mat = np.zeros((nrows * ncols, masks.shape[0]))
    X1, Y1 = np.meshgrid(np.arange(nrows), np.arange(ncols))
    X1, Y1 = np.c_[X1.ravel(), Y1.ravel()].T
    for i, mask in enumerate(masks):

        X2, Y2 = np.nonzero(find_boundaries(mask, mode='inner'))
        sum_x = (X2.reshape(-1, 1) - X1.reshape(1, -1)) ** 2
        sum_y = (Y2.reshape(-1, 1) - Y1.reshape(1, -1)) ** 2
        if len(sum_x) > 0 and len(sum_y) > 0:
            dist_mat[:, i] = np.sqrt(sum_x + sum_y).min(axis=0)
    ix = np.arange(dist_mat.shape[0])
    if dist_mat.shape[1] == 1:
        d1 = dist_mat.ravel()
        border_loss_map = w0 * np.exp((-1 * (d1) ** 2) / (2 * (sigma ** 2)))
    else:
        if dist_mat.shape[1] == 2:
            d1_ix, d2_ix = np.argpartition(dist_mat, 1, axis=1)[:, :2].T
        else:
            d1_ix, d2_ix = np.argpartition(dist_mat, 2, axis=1)[:, :2].T
        d1 = dist_mat[ix, d1_ix]
        d2 = dist_mat[ix, d2_ix]
        border_loss_map = w0 * np.exp((-1 * (d1 + d2) ** 2) / (2 * (sigma ** 2)))
    xBLoss = np.zeros((nrows, ncols))
    xBLoss[X1, Y1] = border_loss_map
    loss = np.zeros((nrows, ncols))
    w_1 = 1 - masks.sum() / loss.size
    w_0 = 1 - w_1
    loss[masks.sum(0) == 1] = w_1
    loss[masks.sum(0) == 0] = w_0
    ZZ = xBLoss + loss
    return ZZ

In [8]:
def normalizedImages(patch_img_batch):
        return (patch_img_batch - np.mean(patch_img_batch))/np.std(patch_img_batch)
    
def getPatch(img_id):
    N_patch = N_split**2
    patch_all = []
    img = getImageAllband(img_id)
    masks = getMask(img_id, (img.shape[0], img.shape[1]))
    for i in range(N_split):
        for j in range(N_split):
            y = masks[Patch_size*i:Patch_size*(i + 1), Patch_size*j:Patch_size*(j + 1)]
            weight_map_y = []
            for mask in y.transpose(2,0,1):
                weight_map_y.append(getWeightMap(np.array([mask])))
            weight_map_y = np.array(weight_map_y).transpose(1, 2, 0)#getWeightMap(y.transpose(2,0,1))
            if np.sum(y) > 0:
                x = img[Patch_size*i:Patch_size*(i + 1), Patch_size*j:Patch_size*(j + 1),:]
                x = normalizedImages(x)
                patch_all.append(np.concatenate((x, y, weight_map_y), axis = 2))
    patch_all = np.asarray(patch_all)
    return patch_all

In [9]:
def saveAllPatches(test_split = 0.2):
    count = 0
    x = []
    img_ids = sorted(grid_sizes_df.IM_ID.unique())
    for i, img_id in enumerate(img_ids):
        print("Saving for image {}".format(img_id))
        x_all = getPatch(img_id)
        if len(x_all) > 0:
            count = count + 1
            if count == 1:
                x = x_all
            else:
                x = np.concatenate((x, x_all), axis = 0)
    try:
        trn = 1 - test_split
        l = len(x)
        train_stump = int(l * trn)
        np.save(processed_data_dir + '/data_pos_%d_%d_train_class%d' % (Patch_size, N_split, class_type), x[:train_stump])
        np.save(processed_data_dir + '/data_pos_%d_%d_test_class%d' % (Patch_size, N_split, class_type), x[train_stump:])
        return None
    except:
        return x

In [ ]:
#x = saveAllPatches()

In [10]:
def getModel(input_shape,
             n_classes,
             epsilon,
             filter_seq = [32, 64, 128, 256, 512],
             dropout_seq = [0.1, 0.2, 0.3, 0.4, 0.5],
             kernel_size = 3,
             activation = 'relu',
             padding = 'same',
             kernel_initializer = 'he_normal'):

    assert len(filter_seq) > 0
    assert len(filter_seq) >= len(dropout_seq)
    if len(filter_seq) > len(dropout_seq):
        pad = len(filter_seq) - len(dropout_seq)
        pad_arr = [None]*pad
        dropout_seq += pad_arr

    def encoderBlock(inp, 
                     filters,
                     kernel_size,
                     block_num,
                     strides = 1,
                     activation = 'relu',
                     padding = 'same',
                     kernel_initializer = 'he_normal',
                     dropout_rate = None):

        conv = Conv2D(filters,
                      kernel_size,
                      strides = strides,
                      activation = activation,
                      padding = padding,
                      kernel_initializer = kernel_initializer,
                      name = 'encoder_layer_{}_1'.format(block_num))(inp)
        conv = Conv2D(filters,
                      kernel_size,
                      strides = strides,
                      activation = activation,
                      padding = padding, 
                      kernel_initializer = kernel_initializer,
                      name = 'encoder_layer_{}_2'.format(block_num))(conv)
        if dropout_rate:
            conv = Dropout(dropout_rate, name = 'enc_dropout_layer_{}'.format(block_num))(conv)
        out = MaxPool2D(name = 'enc_maxpool_layer_{}'.format(block_num))(conv)
        return conv, out

    def decoderBlock(inp, 
                     filters,
                     kernel_size,
                     conv_block,
                     block_num,
                     strides = 1,
                     activation = 'relu',
                     padding = 'same',
                     kernel_initializer = 'he_normal',
                     dropout_rate = None):

        conv = Conv2DTranspose(filters, 
                               2, 
                               strides = 2,
                               padding = padding, 
                               kernel_initializer = kernel_initializer,
                               name = 'dec_layer_transpose_{}'.format(block_num))(inp)
        conv = Concatenate(name = 'dec_concat_layer_{}'.format(block_num))([conv, conv_block])
        conv = Conv2D(filters,
                      kernel_size,
                      activation = activation,
                      padding = padding,
                      strides = strides,
                      kernel_initializer = kernel_initializer,
                      name = 'decoder_layer_{}_1'.format(block_num))(conv)
        conv = Conv2D(filters,
                      kernel_size,
                      activation = activation,
                      padding = padding,
                      strides = strides,
                      kernel_initializer = kernel_initializer,
                      name = 'decoder_layer_{}_2'.format(block_num))(conv)
        if dropout_rate:
            out = Dropout(dropout_rate, name = 'dec_dropout_layer_{}'.format(block_num))(conv)
        return out


    input_layer = Input(shape = input_shape, name = 'main_input_layer')

    weight_map = Input(shape = input_shape[:2] + (n_classes,), name = 'weight_map_input_layer')

    latent_layer_params = (filter_seq[-1], dropout_seq[-1])
    filter_seq = filter_seq[:-1]
    dropout_seq = dropout_seq[:-1]

    # Adding encoder layers
    encoder_blocks_outs = []
    encoder_blocks_conv = []

    for i, (num_filters, dropout_rate)  in enumerate(zip(filter_seq, dropout_seq)):
        if i == 0:
            encoder_conv, encoder_out = encoderBlock(input_layer,
                                                     num_filters,
                                                     kernel_size,
                                                     i+1,
                                                     activation = activation,
                                                     padding = padding,
                                                     kernel_initializer = kernel_initializer,
                                                     dropout_rate = dropout_rate)
        else:
            encoder_conv, encoder_out = encoderBlock(encoder_blocks_outs[i-1],
                                                     num_filters,
                                                     kernel_size,
                                                     i+1,
                                                     activation = activation,
                                                     padding = padding,
                                                     kernel_initializer = kernel_initializer,
                                                     dropout_rate = dropout_rate)
        encoder_blocks_outs.append(encoder_out)
        encoder_blocks_conv.append(encoder_conv)

    # Adding latent layer
    latent_layer = Conv2D(latent_layer_params[0], 
                          3, 
                          activation='relu', 
                          padding='same', 
                          kernel_initializer='he_normal',
                          name = 'latent_layer_1')(encoder_blocks_outs[-1])
    latent_layer = Conv2D(latent_layer_params[0],
                          3, 
                          activation='relu', 
                          padding='same', 
                          kernel_initializer='he_normal',
                          name = 'latent_layer_2')(latent_layer)
    if latent_layer_params[1]:
        latent_layer_out = Dropout(latent_layer_params[1], 
                                   name = 'latent_dropout_layer')(latent_layer)

    decoder_blocks_outs = []

    for i, (num_filters, dropout_rate)  in enumerate(zip(reversed(filter_seq), reversed(dropout_seq))):
        if i == 0:
            decoder_out = decoderBlock(latent_layer_out,
                                       num_filters,
                                       kernel_size,
                                       encoder_blocks_conv[-1-i],
                                       i+1,
                                       activation = activation,
                                       padding = padding,
                                       kernel_initializer = kernel_initializer,
                                       dropout_rate = dropout_rate)
        else:
            decoder_out = decoderBlock(decoder_blocks_outs[i-1],
                                       num_filters,
                                       kernel_size,
                                       encoder_blocks_conv[-1-i],
                                       i+1,
                                       activation = activation,
                                       padding = padding,
                                       kernel_initializer = kernel_initializer,
                                       dropout_rate = dropout_rate)
        decoder_blocks_outs.append(decoder_out)


    # Output softmax layer
    softmax_out = Conv2D(n_classes, 
                         1, 
                         activation = activation, 
                         kernel_initializer = kernel_initializer,
                         name = 'softmax_layer')(decoder_blocks_outs[-1])

    # Adding custom categorical crossentropy loss - Non trainable layers
    

    # Final model
    model = Model(inputs=[input_layer, weight_map], outputs=[softmax_out, weight_map])
        
    return model

In [11]:
def jaccard_coef_int(y_true, y_pred):
    smooth = 1e-12
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)

epsilon = 0.2
def custom_loss(target, output):
    out = output[0]
    wmap = output[1]
    out =  out / tf.reduce_sum(out, len(out.get_shape()) - 1, True)
    out = tf.clip_by_value(out, epsilon, 1. - epsilon)
    out = tf.log(out)
    weighted_output = tf.multiply(out, wmap)
    -tf.reduce_sum(target * output, len(output.get_shape()) - 1)

def dice_coef(y_true, y_pred):
    smooth = 1e-12
    flatten_y_true = K.flatten(y_true)
    flatten_y_pred = K.flatten(y_pred)
    intersection = K.sum(flatten_y_true * flatten_y_pred)
    union = K.sum(flatten_y_true) + K.sum(flatten_y_pred)
    diceCoeff = 2 * (intersection + smooth) / (union + smooth)
    return diceCoeff

In [12]:
input_shape = (224, 224, 20)
n_classes = 1
epsilon = 0
model = getModel(input_shape, n_classes, epsilon)

In [13]:
model.compile(optimizer=Adam(), loss=custom_loss, metrics=[dice_coef, jaccard_coef_int, 'accuracy'])

In [21]:
def get_normalized_patches():
    data = np.load(processed_data_dir + '/data_pos_%d_%d_train_class%d.npy' % (Patch_size, N_split, class_type))
    img = data[:,:,:,:20]
    msk = data[:,:,:,20:21]
    wmap = data[:, :, :, 21:]
    mean = np.mean(img)
    std = np.std(img)
    img = (img - mean)/std
    return img, msk, wmap

In [22]:
img, msk, wmap = get_normalized_patches()

In [23]:
model_checkpoint = ModelCheckpoint('wmap_unet_c1_{epoch:02d}.hdf5')
early_stopping = EarlyStopping(patience = 3)
history = model.fit(x = [img, wmap],
                    y = msk,
                    epochs = 100,
                    batch_size = 64,
                    validation_split = 0.1,
                    callbacks=[model_checkpoint, early_stopping])

Train on 859 samples, validate on 96 samples
Epoch 1/100
859/859 [==============================] - 53s 62ms/step - loss: 0.0000e+00 - dice_coef: 4.9851e-18 - jaccard_coef_int: 5.6420e-16 - accuracy: 0.8596 - val_loss: 0.0000e+00 - val_dice_coef: 3.5131e-18 - val_jaccard_coef_int: 3.9864e-16 - val_accuracy: 0.7164
Epoch 2/100
859/859 [==============================] - 24s 28ms/step - loss: 0.0000e+00 - dice_coef: 5.0123e-18 - jaccard_coef_int: 5.6949e-16 - accuracy: 0.8596 - val_loss: 0.0000e+00 - val_dice_coef: 3.5131e-18 - val_jaccard_coef_int: 3.9864e-16 - val_accuracy: 0.7164
Epoch 3/100
448/859 [==============>...............] - ETA: 10s - loss: 0.0000e+00 - dice_coef: 4.4565e-18 - jaccard_coef_int: 5.0306e-16 - accuracy: 0.8588

KeyboardInterrupt: 

In [24]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input_layer (InputLayer)   (None, 224, 224, 20) 0                                            
__________________________________________________________________________________________________
encoder_layer_1_1 (Conv2D)      (None, 224, 224, 32) 5792        main_input_layer[0][0]           
__________________________________________________________________________________________________
encoder_layer_1_2 (Conv2D)      (None, 224, 224, 32) 9248        encoder_layer_1_1[0][0]          
__________________________________________________________________________________________________
enc_dropout_layer_1 (Dropout)   (None, 224, 224, 32) 0           encoder_layer_1_2[0][0]          
____________________________________________________________________________________________